In [1]:
# import gspread
# from google.colab import auth
# auth.authenticate_user()

# from google.auth import default
# creds, _ = default()
# gc = gspread.authorize(creds)

import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/spreadsheets']
jfile = 'carteira-328314-d38dcc8ee3e4.json'

credentials = ServiceAccountCredentials.from_json_keyfile_name(jfile, scope)
gc = gspread.authorize(credentials)

##import random
#import datetime as dt
from datetime import datetime
import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sb

In [2]:
titulo_planilha = 'Investimentos'
planilha = gc.open(titulo_planilha)
atualiza_saldos = True
pagina = planilha.worksheet('Renda Fixa')

In [3]:

from urllib.request import urlopen

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import json
url = 'https://www.tesourodireto.com.br/json/br/com/b3/tesourodireto/service/api/treasurybondsinfo.json'
response = urlopen(url)
data_json = json.loads(response.read())

In [4]:
nome , pr , pc , tr , tc = [],[],[],[],[]
td=pd.DataFrame()
for i in range(len(data_json["response"]["TrsrBdTradgList"])):
  nome.append(data_json["response"]["TrsrBdTradgList"][i]['TrsrBd']['nm'])
  pr.append(data_json["response"]["TrsrBdTradgList"][i]['TrsrBd']['untrRedVal'])
  pc.append(data_json["response"]["TrsrBdTradgList"][i]['TrsrBd']['untrInvstmtVal'])
  tr.append(data_json["response"]["TrsrBdTradgList"][i]['TrsrBd']['anulRedRate'])
  tc.append(data_json["response"]["TrsrBdTradgList"][i]['TrsrBd']['anulInvstmtRate'])

td['título'] = nome
td['preço resgate'] = pr
td['preço compra'] = pc
td['taxa resgate'] = tr
td['taxa compra'] = tc

td['Ano'] = td['título'].str.extract(r'\b(\d{4})\b')
td = td.sort_values(by=['título','Ano'], ascending=True)
td.drop(['Ano'],axis=1,inplace=True)

td

,título,preço resgate,preço compra,taxa resgate,taxa compra
23,Tesouro Educa+ 2026,3366.20,3382.16,5.6600,5.5400
24,Tesouro Educa+ 2027,3187.27,3206.00,5.6600,5.5400
26,Tesouro Educa+ 2028,3012.23,3033.34,5.6900,5.5700
27,Tesouro Educa+ 2029,2845.21,2868.39,5.7200,5.6000
28,Tesouro Educa+ 2030,2687.65,2712.60,5.7400,5.6200
31,Tesouro Educa+ 2031,2535.78,2562.21,5.7700,5.6500
32,Tesouro Educa+ 2032,2395.86,2423.57,5.7800,5.6600
33,Tesouro Educa+ 2033,2263.36,2292.12,5.7900,5.6700
34,Tesouro Educa+ 2034,2138.07,2167.67,5.8000,5.6800
35,Tesouro Educa+ 2035,2019.30,2049.55,5.8100,5.6900


In [5]:
from datetime import date
today = date.today().strftime('%d/%m/%Y')

In [6]:
linha = 74#int(input('linha da celula inicial (74)'))
pagina.update_cell(linha, 1,  today )

{'spreadsheetId': '1Mse441x9H8uByQkSTVngIKUmH5iBFwEuhGjo1LhqGE8',
 'updatedRange': "'Renda Fixa'!A74",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [7]:

pagina.update('a'+str(linha+1),[td.columns.values.tolist()] + td.values.tolist())

/tmp/ipykernel_7521/1005078071.py:1: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  pagina.update('a'+str(linha+1),[td.columns.values.tolist()] + td.values.tolist())


{'spreadsheetId': '1Mse441x9H8uByQkSTVngIKUmH5iBFwEuhGjo1LhqGE8',
 'updatedRange': "'Renda Fixa'!A75:E132",
 'updatedRows': 58,
 'updatedColumns': 5,
 'updatedCells': 290}

In [8]:
meus_tits = []
for linha in range(3,11):
    meus_tits.append(pagina.cell(linha, 1).value)
meus_tits

['Tesouro IPCA+ 2035',
 'Tesouro IPCA+ 2045',
 'Tesouro IPCA+ com Juros Semestrais 2032',
 'Tesouro IPCA+ com Juros Semestrais 2040',
 'Tesouro IPCA+ com Juros Semestrais 2055',
 'Tesouro Prefixado 2025',
 'Tesouro Prefixado 2029',
 'Tesouro IPCA+ 2026']

In [9]:
meus = td[td['título'].isin(meus_tits)].copy()
meus['título']= meus['título'].str.replace('Tesouro ', '')
meus['título']= meus['título'].str.replace('com Juros Semestrais', 'Juros')
meus['título']= meus['título'].str.replace('20', '')

meus = meus.rename(columns={'título':str(today)})

meus

,14/03/2024,preço resgate,preço compra,taxa resgate,taxa compra
19,IPCA+ 26,3712.25,0.00,5.71,0.00
29,IPCA+ 35,2270.62,2299.41,5.80,5.68
42,IPCA+ 45,1270.61,1301.29,5.90,5.78
25,IPCA+ Juros 32,4339.00,0.00,5.76,0.00
36,IPCA+ Juros 40,4366.16,4419.59,5.80,5.68
49,IPCA+ Juros 55,4444.33,4518.04,5.84,5.72
6,Prefixado 25,927.45,0.00,9.80,0.00
11,Prefixado 29,618.99,0.00,10.57,0.00


In [10]:
pagina.update_cell(2, 12,  today )

pagina.update('l2',[meus.columns.values.tolist()] + meus.values.tolist())

/tmp/ipykernel_7521/2700457520.py:3: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  pagina.update('l2',[meus.columns.values.tolist()] + meus.values.tolist())


{'spreadsheetId': '1Mse441x9H8uByQkSTVngIKUmH5iBFwEuhGjo1LhqGE8',
 'updatedRange': "'Renda Fixa'!L2:P10",
 'updatedRows': 9,
 'updatedColumns': 5,
 'updatedCells': 45}